This notebook reads through the data for all the buoys and plots the relevant figures. As done for a single buoy, we will first import some requisite modules. 

In [10]:
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plots
import readsimdata
import settings
import generalfunc as gf 
import os

In [11]:
#plotting parameters.
params = { "pgf.texsystem": "pdflatex",
           "pgf.preamble": [ r"\usepackage[utf8x]{inputenc}",
                             r"\usepackage[T1]{fontenc}",
                             r"\usepackage{cmbright}"],
            'legend.fontsize': '10',
            'axes.labelsize': '11',
            'axes.titlesize':'11',
            'xtick.labelsize':'11',
            'ytick.labelsize':'11',
            "font.weight":'bold'}
plt.rcParams.update(params)

At first we try to plot all the observed buoy tracks on a single plot. For this we need to read the data from all the buoys stored in the folder named $container.$

In [12]:
importlib.reload(readsimdata)
folder='../../container/Buoy_data_excel/procesd_buoy_loc_data_Lars'
BD=readsimdata.readallbuoy(folder)

In [13]:
importlib.reload(plots)
locname='Allbuoys'
loc='../../generated_data/'+locname
gf.mkdir_p(loc)
# plots.pltallbuoytracks(loc,BD)

Allbuoys directory already exists.


Now the next plotting is about graphs for all the buoys and different forcings. Here we plot the amplitude and phase of the residual velocity vectors at the different tidal frequencies for all the buoys with different forcings.

In [8]:
def computedict(folder,barray,typ,tidindex):
    #loc means generated data folder.
    tidval={} #tid is the type of tide like M2    
    for b in barray:
        SD=readsimdata.readsimdata(b,folder)
        if typ=='buoy':
            AmvecU=SD['BAmpU'];PhvecU=SD['BPhU']
            AmvecV=SD['BAmpV'];PhvecV=SD['BPhV']
            tidvalft=np.array([AmvecU[tidindex],PhvecU[tidindex],AmvecV[tidindex],PhvecV[tidindex]])
        elif typ=='sim':
            AmvecU=SD['SAmpU'];PhvecU=SD['SPhU']
            AmvecV=SD['SAmpV'];PhvecV=SD['SPhV']
            tidvalft=np.array([AmvecU[tidindex],PhvecU[tidindex],AmvecV[tidindex],PhvecV[tidindex]])
        tidval.update({b:tidvalft})
    return(tidval)

In [15]:
basefol='h1f1A1T1O1Po1Pt1Bf0'
barray=['02','03','09','13','14','16'] 
tide='M2';tidindex=2
btidval=computedict(basefol,barray,'buoy',tidindex)
stidval=computedict(basefol,barray,'sim',tidindex)
typ=['no_coriolis','no_air','no_tidal','no_ocean','no_pgocean','no_pgtidal']
# typ=['cor+pgs+bf+air']
# typ=['no_air']
on='1';off='0'
# bname='BUOY_'+b
for t in range(len(typ)):
    ofv=[on,on,on,on,on,on,on]
    if typ[t]=='cor+pgs':
        ofv[1]=ofv[2]=ofv[3]=ofv[4]=ofv[6]=off
    elif typ[t]=='cor+pgs+bf':
        ofv[1]=ofv[2]=ofv[3]=ofv[4]=off
    elif typ[t]=='cor+pgs+bf+air':
        ofv[2]=ofv[3]=ofv[4]=off
    else:
        ofv[t]=off;ofv[6]=off
    compfol='h1'+'f'+ofv[0]+'A'+ofv[1]+'T'+ofv[2]+'O'+ofv[3]+'Po'+ofv[4]+'Pt'+ofv[5]+'Bf'+ofv[6]
    ftidval=computedict(compfol,barray,'sim',tidindex)
    ftyp=typ[t]
    gf.mkdir_p(loc+'/'+ftyp)    
    plottidal(tide,ftyp,loc,barray,btidval,stidval,ftidval)

no_coriolis directory created
U_Amplitude (m/s)
[0.11129794 0.15245395 0.09877526 0.08262318 0.27716573 0.20448669] [0.07808554 0.13312933 0.08319742 0.06327474 0.26177156 0.16392376] [0.0752412  0.12361683 0.07588577 0.05077246 0.23624982 0.15737397]
U_Phase (deg)
[-141.72255485  -64.7612983  -113.03780655  -66.48364913  -26.16397569
  -68.6338441 ] [-166.05649749  -93.61819839 -142.53583515  -78.0605413   -50.26563091
  -97.1723884 ] [-173.61016487 -101.98893801 -153.0030661   -84.58325089  -56.32129491
 -105.11787926]
V_Amplitude (m/s)
[0.09542751 0.13808085 0.10526378 0.09481807 0.19137874 0.16928929] [0.06609088 0.11273061 0.08921476 0.07515547 0.18584152 0.126634  ] [0.06721352 0.11536664 0.09039167 0.08010146 0.20622201 0.12615422]
V_Phase (deg)
[-57.73574638  14.69466907 -33.33549021 -10.2204829   30.34036539
  21.21509718] [-87.48301475 -17.36494194 -66.03334548 -40.34609707   2.55162386
 -11.11655799] [-97.75047112 -29.33585373 -75.11137167 -45.43147189  -6.96067871
 -24.2975

In [6]:
def plottidal(tide,ftyp,loc,barray,btidval,stidval,ftidval):
    btidvec=np.array(list(btidval.values()))
    stidvec=np.array(list(stidval.values()))
    ftidvec=np.array(list(ftidval.values()))
    namevec=['U_Amplitude (m/s)','U_Phase (deg)','V_Amplitude (m/s)','V_Phase (deg)']
    for n in range(len(namevec)):
        name=namevec[n]
        fig=plt.figure(figsize=(12,7),frameon=True)      
        plt.plot(barray,btidvec[:,n], marker='o', linestyle='dashed',
                 linewidth=2, markersize=12,label='observed')
        plt.plot(barray,stidvec[:,n], marker='o', linestyle='dashed',
                 linewidth=2, markersize=12,label='model')
        plt.plot(barray,ftidvec[:,n], marker='o', linestyle='dashed',
                 linewidth=2, markersize=12,label=ftyp)
        plt.xlabel('Buoys',fontsize=16,fontweight='bold')
        plt.ylabel(name,fontsize=16,fontweight='bold')
        print(name)
        print(btidvec[:,n],stidvec[:,n],ftidvec[:,n])
        plt.legend(loc=1)
#         plt.title(title,fontsize=18,fontweight='bold')
        plt.savefig(loc+'/'+ftyp+'/'+ftyp+tide+name[0:4]+'.jpg',format='jpg')
        plt.close(fig)
